In [1]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [15]:
!pip install --upgrade tables

Requirement already up-to-date: tables in c:\users\abhishek\anaconda3\lib\site-packages (3.4.4)


In [26]:
import pandas as pd
import numpy as np
from nltk.sentiment import SentimentAnalyzer
import unicodedata
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [27]:
#Path here

#Path here

base_path = 'data/created/'
sp500_path = '/data/sp/^GSPC.csv'
created_data_path = base_path + 'data500.h5'
created_data_path_new = base_path + 'data500_new.h5'
technical500_path = base_path + 'technical500.h5'
news_path = 'data/news/'

In [28]:

import pandas as pd
data = pd.read_hdf(created_data_path);

In [29]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Abhishek\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [30]:
data.head()

,Open,High,Low,Close,Adj Close,Volume,lrets,MACD,stochastics,ATR
Date,,,,,,,,,,
1990-01-04,358.760010,358.760010,352.890015,355.670013,355.670013,177000000,-0.008650,0.000000,NaN,5.869995
1990-01-05,355.670013,355.670013,351.350006,352.200012,352.200012,158530000,-0.009804,-0.036878,NaN,4.320007
1990-01-08,352.200012,354.239990,350.540009,353.790009,353.790009,140110000,0.004504,-0.005628,NaN,3.699981
1990-01-09,353.829987,354.170013,349.609985,349.619995,349.619995,155210000,-0.011857,-0.100883,NaN,4.560028
1990-01-10,349.619995,349.619995,344.320007,347.309998,347.309998,175990000,-0.006629,-0.198862,NaN,5.299988


# Remove data first day

In [31]:
data = data.loc["1990-01-02":]
data.head()

,Open,High,Low,Close,Adj Close,Volume,lrets,MACD,stochastics,ATR
Date,,,,,,,,,,
1990-01-04,358.760010,358.760010,352.890015,355.670013,355.670013,177000000,-0.008650,0.000000,NaN,5.869995
1990-01-05,355.670013,355.670013,351.350006,352.200012,352.200012,158530000,-0.009804,-0.036878,NaN,4.320007
1990-01-08,352.200012,354.239990,350.540009,353.790009,353.790009,140110000,0.004504,-0.005628,NaN,3.699981
1990-01-09,353.829987,354.170013,349.609985,349.619995,349.619995,155210000,-0.011857,-0.100883,NaN,4.560028
1990-01-10,349.619995,349.619995,344.320007,347.309998,347.309998,175990000,-0.006629,-0.198862,NaN,5.299988


# Remove holiday 
Feel free to use forward fill to replace Nan value

In [32]:
data.dropna(axis=0, inplace=True)
data.head()

,Open,High,Low,Close,Adj Close,Volume,lrets,MACD,stochastics,ATR
Date,,,,,,,,,,
1990-01-23,330.380005,332.760010,328.670013,331.609985,331.609985,179300000,0.003716,-1.021436,4.863502,4.089997
1990-01-24,331.609985,331.709991,324.170013,330.260010,330.260010,207830000,-0.004079,-1.103580,0.000000,7.539978
1990-01-25,330.260010,332.329987,325.329987,326.079987,326.079987,172270000,-0.012738,-1.323376,0.000000,7.000000
1990-01-26,326.089996,328.579987,321.440002,325.799988,325.799988,198190000,-0.000859,-1.419756,0.000000,7.139985
1990-01-29,325.799988,327.309998,321.790009,325.200012,325.200012,150770000,-0.001843,-1.447355,0.000000,5.519989


# Removing open high low close

In [33]:
#data.drop(['close', 'high', 'low', 'open','adj_factor', 'adj_high', 'adj_low', 'adj_open', 'adj_volume', 'ex_dividend', 'split_ratio'], axis=1, inplace=True)


data.drop(['Open', 'High', 'Low', 'Close'], axis=1, inplace=True)
data.head()

,Adj Close,Volume,lrets,MACD,stochastics,ATR
Date,,,,,,
1990-01-23,331.609985,179300000,0.003716,-1.021436,4.863502,4.089997
1990-01-24,330.260010,207830000,-0.004079,-1.103580,0.000000,7.539978
1990-01-25,326.079987,172270000,-0.012738,-1.323376,0.000000,7.000000
1990-01-26,325.799988,198190000,-0.000859,-1.419756,0.000000,7.139985
1990-01-29,325.200012,150770000,-0.001843,-1.447355,0.000000,5.519989


# Using NLTK sentiment analyzer to generate a polarity score

Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

https://www.nltk.org/_modules/nltk/sentiment/sentiment_analyzer.html

In [34]:
def sentimentanalysis(df):
    sid = SentimentIntensityAnalyzer()
    for date in df.index:
        try:
            sentence = unicodedata.normalize('NFKD', df.loc[date, 'News'])
            ss = sid.polarity_scores(sentence)
            neg_val = ss['neg']
            neu_val = ss['neu']
            pos_val = ss['pos']
            df.at[date, 'neg'] = neg_val
            df.at[date, 'neu'] = neu_val
            df.at[date, 'pos'] = pos_val
            
        except TypeError:
            print (df.loc[date, 'News'])
            print (date)
    return df

In [35]:
new_data = sentimentanalysis(data)

KeyError: 'the label [News] is not in the [index]'

In [14]:
new_data.drop(['News'], axis=1, inplace=True)

NameError: name 'new_data' is not defined

In [0]:
#new_data.to_hdf('data2', 'new_data')
new_data.to_hdf(created_data_path_new, 'new_data')

# Check if there is missing data

In [16]:
new_data.isnull().any()

Adj Close      False
Volume         False
lrets          False
MACD           False
stochastics    False
ATR            False
neg            False
neu            False
pos            False
dtype: bool